Sumber Kode Artikel : https://www.geeksforgeeks.org/machine-learning/pseudo-labelling-semi-supervised-learning/

## Library

In [217]:
import pandas as pd
import numpy as np
import re
import html
import string
from sklearn.datasets import load_digits
from sklearn.model_selection import train_test_split
from sklearn.ensemble import RandomForestClassifier
from sklearn.metrics import accuracy_score
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.linear_model import LogisticRegression

## Preparation Data

In [218]:
pd.set_option('display.max_colwidth', None)
df = pd.read_csv("data_training.csv", delimiter=";")
df

,Label,Comment
0,Negatif,Jujur statement dia ini kek sok idealis gitu ya nggak sih
1,Positif,"baru nonton film nya second lead yg cewek suka banget. gak ada yg salah sama film nya. bagus kok. karakter abizar gak dibuat sok ganteng, senatural mungkin."
2,Negatif,lebih banyak penonton masterpiece merahputih one for all
3,Negatif,"kok bisa nggak ngomenin perannya si ardhito yg jdi sekretaris cha, anjir itu gk tau lah jauh bgt yg jauhnya sampe gk ketulung lagi mulai dari karakter, terus penampilan yg minimal gitu ya jangan kayak jojon­ cara jalan dan bajunya itulohhh"
4,Positif,"Part sin hari saat tanya sama temo masalah kenapa takut hujan, dan sin hari memberi dukungan yang beda dari orang orang. Ini juga sekedarnya malah pas di film ini. Sekedae sari tanya sama Tomo bab takut hujan, abis itu malah tanya hubungan sama satrio. Njir parah dah. Padahal part ini salah satu yang bikin temo tambah suka sama sin hari"
...,...,...
255,Netral,Aku dah nonton Drakor nya emang seru dan lucu tapi gak minat nonton yang versi indo ini
256,Netral,Saya kasian sama pemilik film dan aktor lain nya yang kena imbasnya gara2 satu orang.
257,Negatif,"Star syndrom nih anak karna dpt peran utama, dikira dah selevel Reza Rahardian atau Nicholas Saputra kalee"
258,Negatif,Emang tolol sih si abidzar Kalo bukan gara2 dia pasti rame yg nonton


## Melihat Jumlah Per Label

In [219]:
jumlah = df["Label"].value_counts()
jumlah

,count
Label,
Negatif,110
Netral,110
Positif,40


## Melihat 10 Label Positif

In [220]:
sent_positif = df[df["Label"] == "Positif"].head(10)
sent_positif

,Label,Comment
1,Positif,"baru nonton film nya second lead yg cewek suka banget. gak ada yg salah sama film nya. bagus kok. karakter abizar gak dibuat sok ganteng, senatural mungkin."
4,Positif,"Part sin hari saat tanya sama temo masalah kenapa takut hujan, dan sin hari memberi dukungan yang beda dari orang orang. Ini juga sekedarnya malah pas di film ini. Sekedae sari tanya sama Tomo bab takut hujan, abis itu malah tanya hubungan sama satrio. Njir parah dah. Padahal part ini salah satu yang bikin temo tambah suka sama sin hari"
19,Positif,Ga seburuk itu si filmny
27,Positif,Tapi jujur film nya seru weh
32,Positif,"Gw sangat setuju dengan statement lu bro, lu ga bisa kontrol masyarakat atau opini masyarakat, jadi yang bisa lu kontrol adalah omongan sendiri"
34,Positif,"Paling enggak episode terakhirnya bagus animasinya, jauh lebih bagus gambaran di komiknya sih"
35,Positif,"Harusnya kalo idealis dengan ketidaksukaan meniru karakter yang sudah ada, dan ingin pake karakter sendiri. Pertama, gausah ambil jobnya, kedua, mininal jangan film yang karakternya dah pakem tentang kepribadian, kaya yang dah terjelaskan di novel, film, atau apapun."
56,Positif,Trgetmya bukan cuma Fans kpopnya tapi yg non kpopers juga makanya dibuat versi indonya agar non kpopers juga tertarik
61,Positif,"Mnurutku mereka pilih Ariel sama Abizar itu karna pengen buat yang melokal aja gitu bg, yang Indonesia bgt mukanya. Nah tp aku bingung nih di sini, kalo kita buat film adaptasi dari negara org lain itu boleh kita ubah atau ga boleh diubah ya?"
114,Positif,Filmya sih ringan dan cukup menghibur sih. Untuk bikin ketawa. Tapi kena cancel duluan. Jadi kasihan


## Melihat 10 Label Negatif

In [221]:
sent_negatif = df[df["Label"] == "Negatif"].head(10)
sent_negatif

,Label,Comment
0,Negatif,Jujur statement dia ini kek sok idealis gitu ya nggak sih
2,Negatif,lebih banyak penonton masterpiece merahputih one for all
3,Negatif,"kok bisa nggak ngomenin perannya si ardhito yg jdi sekretaris cha, anjir itu gk tau lah jauh bgt yg jauhnya sampe gk ketulung lagi mulai dari karakter, terus penampilan yg minimal gitu ya jangan kayak jojon­ cara jalan dan bajunya itulohhh"
5,Negatif,dia nongol di netflix aja males gua nontonnya
9,Negatif,Abizar jangan sok jago
10,Negatif,"Namanya aktor ya harus bs meranin karakter sesuai skenario. Nape itu orang sosoan mau munculin karakternya sendiri? Ga usah main pelem brarti, ngevlog aja ato mukbang"
11,Negatif,Orang prindavan di ajak maen film
12,Negatif,"Bahkan gw yang fans anime pun, ikut muak dengan aktor ini"
15,Negatif,sperti biasa yg nga trima fans para drakor norak asu
16,Negatif,Lo boleh sombong kalo lo emang sejago itu. Kalo masih rempeyek ya jgn Aneh aneh dulu


## Melihat 10 Label Netral

In [222]:
sent_netral = df[df["Label"] == "Netral"].head(10)
sent_netral

,Label,Comment
6,Netral,Gw baru tau rule 4 itu btw gw suka nonton anime apakah masuk ke kategori ke 2
7,Netral,Sekarang sedikit naik 1.8 tapi tetap jelek juga
8,Netral,"Ibarat film naruto, pngn anda remake.. naruto nya anda buat characternya cool"
13,Netral,"Gilaaaaa, bisa bisanya ada orang yang ngeludahin target konsumen film yang dia bintangi"
14,Netral,"Tapi sempat di nomor satu di Netflix film Business Proposal ini. Terakhir kali aku lihat di Netflix, waktu aku mau nonton film horor Indonesia di Netflix. Enggak tahu sekarang posisi di mana?"
18,Netral,Intinya kalau baru sekali main film jgn sombong itu aja intinya.
20,Netral,Ngga seburuk itu Dan ngga se bagus itu juga
22,Netral,Jujur Gue Liat Abizar Kurang Respect Gayanya tengil. Agak Sombong. Apalagi Melihat Background Ortunya. Sangat Di Sayangkan. Gk Profesional Dan Gk Serius. Ngerasa Udah Super Star
25,Netral,"Flem in sebener nya bagus, untuk netizen biasa, tapi untuk yg sok pintar pasti bnyk di permsalhin"
29,Netral,Kasian sama pemain lain yang bener bener niat


## Preprocessing

In [223]:
def preprocess(text):
    text = text.lower()
    text = re.sub(r'[^\w\s]', '', text)
    text = re.sub(r'\s+', ' ', text).strip()
    return text

In [224]:

df["Comment"] = df["Comment"].apply(preprocess)
df["Comment"]

,Comment
0,jujur statement dia ini kek sok idealis gitu ya nggak sih
1,baru nonton film nya second lead yg cewek suka banget gak ada yg salah sama film nya bagus kok karakter abizar gak dibuat sok ganteng senatural mungkin
2,lebih banyak penonton masterpiece merahputih one for all
3,kok bisa nggak ngomenin perannya si ardhito yg jdi sekretaris cha anjir itu gk tau lah jauh bgt yg jauhnya sampe gk ketulung lagi mulai dari karakter terus penampilan yg minimal gitu ya jangan kayak jojon cara jalan dan bajunya itulohhh
4,part sin hari saat tanya sama temo masalah kenapa takut hujan dan sin hari memberi dukungan yang beda dari orang orang ini juga sekedarnya malah pas di film ini sekedae sari tanya sama tomo bab takut hujan abis itu malah tanya hubungan sama satrio njir parah dah padahal part ini salah satu yang bikin temo tambah suka sama sin hari
...,...
255,aku dah nonton drakor nya emang seru dan lucu tapi gak minat nonton yang versi indo ini
256,saya kasian sama pemilik film dan aktor lain nya yang kena imbasnya gara2 satu orang
257,star syndrom nih anak karna dpt peran utama dikira dah selevel reza rahardian atau nicholas saputra kalee
258,emang tolol sih si abidzar kalo bukan gara2 dia pasti rame yg nonton


## Train Test Split

In [225]:
vectorizer = TfidfVectorizer()

X = df["Comment"]
y = df["Label"]

# Split into labeled and unlabeled data (20% labeled, 80% unlabeled)
X_labeled, X_unlabeled, y_labeled, y_unlabeled = train_test_split(
    X, y, test_size=0.95, stratify=y, random_state=42
)

# Further split a part of unlabeled data as validation set
X_unlabeled, X_val, y_unlabeled, y_val = train_test_split(
    X_unlabeled, y_unlabeled, test_size=0.25, stratify=y_unlabeled, random_state=42
)


## Pseudo Labelling Function

In [226]:
def pseudo_labeling(X_labeled, y_labeled, X_unlabeled, model, threshold=0.95, max_iters=5):
    for iteration in range(max_iters):
        print(f"\nIteration {iteration + 1}")

        # Train model on labeled data
        X_labeled = vectorizer.fit_transform(X_labeled)
        model.fit(X_labeled, y_labeled)

        # Predict on unlabeled data
        X_unlabeled = vectorizer.transform(X_unlabeled)
        probs = model.predict_proba(X_unlabeled)
        preds = np.argmax(probs, axis=1)
        confidences = np.max(probs, axis=1)

        # Select high-confidence pseudo-labels
        mask = confidences >= threshold
        X_pseudo = X_unlabeled[mask]
        y_pseudo = preds[mask]

        if X_pseudo.shape[0] == 0:
            print("No high-confidence pseudo-labels found. Stopping.")
            break

        # Combine with original labeled data
        X_labeled = np.vstack([X_labeled, X_pseudo])
        y_labeled = np.concatenate([y_labeled, y_pseudo])

        # Remove pseudo-labeled examples from unlabeled set
        X_unlabeled = X_unlabeled[~mask]

        print(f"Added {len(X_pseudo)} pseudo-labeled samples.")

    return model

## Training Model

In [227]:
# Initialize model
model = RandomForestClassifier(n_estimators=100)

# Run pseudo-labelling
final_model = pseudo_labeling(X_labeled, y_labeled, X_unlabeled, model, threshold=0.95)

# Evaluate on held-out validation set
X_val = vectorizer.transform(X_val)
y_pred = final_model.predict(X_val)
print("\nValidation Accuracy:", accuracy_score(y_val, y_pred))


Iteration 1
No high-confidence pseudo-labels found. Stopping.

Validation Accuracy: 0.5161290322580645


## Implementasi Pseudo Labelling

In [228]:
no_labelling_data = pd.read_csv("no_label_dataset.csv")
no_labelling_data.head(5)

,Comment
0,belakangan intro ngelantur sangat keren
1,adaptasi drakor kenapa jadi macem adaptasi film prindapan fixsalah casting ini mah abijar cocok kalo meranin adaptasi kabhi kushi kabhi gam
2,dasar fans ribet
3,dulu waktu gw ada drama pentas buat ujian sekolah dan gw meranin tokoh utama sejarah suatu kota gw sebenernya pendatang di kota itu dan gak tau sama sekali soal tokoh itu terus gw research di internet berharihari setiap latihan karena gw tahu yang nonton gw nanti udah tahu tentang karakter yang gw mainin sedangkan gw gak tau sama sekali setelah sebulanan latihan gw berhasil paham karakternya dan pas tampil berhasil dapet pujian dari guru gw dan sebenarnya ada sedikit improvisasi yang gw lakukan di karakter yang gw mainin tapi gw gak mengkhianati sumber aslinya yang pasti jaga sikap dengan penonton yang akan menontonmu karena jika kamu mendapat peran besar maka tanggung jawabmu juga besar
4,itu kalo anda mau belajar lha a lulus aja kagak paket c maunya instan


In [229]:
no_labelling_data = no_labelling_data.dropna(subset=['Comment'])

In [230]:
X_no_labelled = vectorizer.transform(no_labelling_data['Comment'])

In [231]:
y_pred = final_model.predict(X_no_labelled)

In [245]:
no_labelling_data['Label_Prediksi'] = y_pred

result = no_labelling_data[['Comment', 'Label_Prediksi']]

In [246]:
result.to_csv("hasil-prediksi-manual.csv")
print("Berhasil Menyimpan")